In [20]:
from crewai import Agent, Crew, Process, Task
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

In [21]:
llm = ChatOpenAI(model = "gpt-4o", temperature = 0.5)

In [22]:
# Call for inputs: Interviewer, company, job position, job description:
interviewer = input("Enter the name of the interviewer (if unknown include None): ")
company = input("Enter the name of the company: ")
job_position = input("Enter the job position: ")
job_description = input("Enter the job description: ")

In [23]:
research_agent = Agent(
    role = "Research Agent",
    goal = "conduct in-depth research",
    backstory = """ As a Research Specialist, your mission is to uncover detailed and relevant information.
    The ultimate goal is to meet the expectations o the task""",
    llm = llm,
    max_iter = 3,
    tools = [SerperDevTool()]
)

In [24]:
research_company_task = Task(
    description=f"""
    Research {company} and return:
    - Exactly 5 bullet points about the company
    - Exactly 3 bullet points about industry trends (last 12 months)
    - Each bullet max 15 words
    - Include one source URL per bullet
    Stop once complete.
    """,
    expected_output="""
    Markdown with exactly 8 bullet points.
    Each bullet <= 15 words and includes a source URL.
    """,
    agent=research_agent
)

In [25]:
# Research Interviewer Task
research_person_task = Task(
    description=f"""
    Research {interviewer} and return:
    - Exactly 5 bullet points
    - Each bullet max 15 words
    - Include one source URL
    Stop once complete.
    """,
    expected_output="""
    Markdown with exactly 5 bullet points.
    Each bullet <= 15 words and includes a source URL.
    """,
    agent=research_agent
)


In [26]:
# Build the AI interview coach agent
coach_agent = Agent(
    role = "Interview Coach",
    goal = f"I help the user prepare for the job interview for job {job_position}",
    backstory = f"You are an expert in job interviews in companies like {company}",
    max_iter = 3,
    llm = llm
)

In [27]:
# Prepare the lsit of questions
define_questions_task = Task(
    description = f"""
    Define a list a of questions to prepare the user for the job {job_position} based on the company {company}, job description {job_description} and interviewer {interviewer}""",
    expected_output = f""" a list of 20 questions to ask the user in Markdown format that cover: Culture and Team fit, Job position Fit, Background and ways of working, Growth Mindset""",
    agent = coach_agent,
    context = [research_company_task, research_person_task]
)

In [28]:
# Assembling the crew
crew = Crew(
    agents = [research_agent, coach_agent],
    tasks = [research_company_task, research_person_task, define_questions_task],
    verbose = True,
    process = Process.sequential
)
result = crew.kickoff({"topic": "Write a list of question to prepare for the the interview"}
)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 02fecc62-1e3a-4bc6-b384-8db3bc154f19                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Research Agent                                                                                          │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Research Facebook and return:                                                                              │
│      - Exactly 5 bullet points about the company                                                                │
│      - Exactly 3 bullet points about industry trends (last 12 months)                                           │
│      - Each bullet max 15 words                                                                                 │
│      - Include one source URL per bullet                                                                        │
│      Stop once complete.                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/Users/dylanthunn/.local/share/virtualenvs/agents-0RAN7L6M/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')


/Users/dylanthunn/.local/share/virtualenvs/agents-0RAN7L6M/lib/python3.12/site-packages/rich/live.py:256: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Research Agent                                                                                          │
│                                                                                                                 │
│  Thought: Thought: I need to gather information about Facebook and recent industry trends. I will start by      │
│  searching for information about Facebook.                                                                      │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

/Users/dylanthunn/.local/share/virtualenvs/agents-0RAN7L6M/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')


╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Interview Coach                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  ### Culture and Team Fit                                                                                       │
│  1. How do you align with Meta's mission to build the future of human connection?                               │
│  2. Can you share an experience where you contributed to a team project that had a significant impact?          │
│  3. How do you adapt to working in a fast-paced and constantly changing environment like Facebook?              │
│  4. How do you handle conflicts in a team setting?                                                              │
│  5. What does inclusivity mean to you, and how do you foster an inclusive environment in your work?             │
│                                                                                                                 │
│  ### Job Position Fit                                                                                           │
│  6. How would you leverage your skills to contribute to Facebook's growth in areas like social networking and   │
│  advertising?                                                                                                   │
│  7. What excites you about working on video content and AI-generated content for social media platforms?        │
│  8. How would you approach developing more immersive AR and VR experiences for Facebook users?                  │
│  9. Can you discuss a project where you had to innovate to solve a problem?                                     │
│  10. How do you prioritize tasks when faced with multiple deadlines?                                            │
│                                                                                                                 │
│  ### Background and Ways of Working                                                                             │
│  11. How has your previous experience prepared you for a role at Facebook?                                      │
│  12. Can you describe a time when you had to learn a new technology quickly?                                    │
│  13. What role do you typically play in team projects, and how do you ensure success?                           │
│  14. How do you stay updated with the latest trends in technology and social media?                             │
│  15. Describe a challenging project you worked on and how you overcame the obstacles.                           │
│                                                                                                                 │
│  ### Growth Mindset                                                                                             │
│  16. How do you approach personal and professional growth?                                                      │
│  17. Can you provide an example of a time you received constructive criticism and how you applied it?           │
│  18. What motivates you to continuously improve in your field?                                                  │
│  19. How do you set goals for yourself, and what strategies do you use to achieve them?                         │
│  20. Describe a situation where you had to step outside your comfort zone to achieve a goal.                    │
│  ```                                                   

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 401c95b0-5cfe-4044-9dc4-a201316dd10c                                                                     │
│  Agent: Interview Coach                                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 02fecc62-1e3a-4bc6-b384-8db3bc154f19                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```markdown                                                                                      │
│  ### Culture and Team Fit                                                                                       │
│  1. How do you align with Meta's mission to build the future of human connection?                               │
│  2. Can you share an experience where you contributed to a team project that had a significant impact?          │
│  3. How do you adapt to working in a fast-paced and constantly changing environment like Facebook?              │
│  4. How do you handle conflicts in a team setting?                                                              │
│  5. What does inclusivity mean to you, and how do you foster an inclusive environment in your work?             │
│                                                                                                                 │
│  ### Job Position Fit                                                                                           │
│  6. How would you leverage your skills to contribute to Facebook's growth in areas like social networking and   │
│  advertising?                                                                                                   │
│  7. What excites you about working on video content and AI-generated content for social media platforms?        │
│  8. How would you approach developing more immersive AR and VR experiences for Facebook users?                  │
│  9. Can you discuss a project where you had to innovate to solve a problem?                                     │
│  10. How do you prioritize tasks when faced with multiple deadlines?                                            │
│                                                                                                                 │
│  ### Background and Ways of Working                                                                             │
│  11. How has your previous experience prepared you for a role at Facebook?                                      │
│  12. Can you describe a time when you had to learn a new technology quickly?                                    │
│  13. What role do you typically play in team projects, and how do you ensure success?                           │
│  14. How do you stay updated with the latest trends in technology and social media?                             │
│  15. Describe a challenging project you worked on and how you overcame the obstacles.                           │
│                                                                                                                 │
│  ### Growth Mindset                                                                                             │
│  16. How do you approach personal and professional growth?                                                      │
│  17. Can you provide an example of a time you received constructive criticism and how you applied it?           │
│  18. What motivates you to continuously improve in your field?                                                  │
│  19. How do you set goals for yourself, and what strategies do you use to achieve them?                         │
│  20. Describe a situation where you had to step outsid

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [29]:
# Check the crew outcome
display(Markdown(result.tasks_output[2].raw))

```markdown
### Culture and Team Fit
1. How do you align with Meta's mission to build the future of human connection?
2. Can you share an experience where you contributed to a team project that had a significant impact?
3. How do you adapt to working in a fast-paced and constantly changing environment like Facebook?
4. How do you handle conflicts in a team setting?
5. What does inclusivity mean to you, and how do you foster an inclusive environment in your work?

### Job Position Fit
6. How would you leverage your skills to contribute to Facebook's growth in areas like social networking and advertising?
7. What excites you about working on video content and AI-generated content for social media platforms?
8. How would you approach developing more immersive AR and VR experiences for Facebook users?
9. Can you discuss a project where you had to innovate to solve a problem?
10. How do you prioritize tasks when faced with multiple deadlines?

### Background and Ways of Working
11. How has your previous experience prepared you for a role at Facebook?
12. Can you describe a time when you had to learn a new technology quickly?
13. What role do you typically play in team projects, and how do you ensure success?
14. How do you stay updated with the latest trends in technology and social media?
15. Describe a challenging project you worked on and how you overcame the obstacles.

### Growth Mindset
16. How do you approach personal and professional growth?
17. Can you provide an example of a time you received constructive criticism and how you applied it?
18. What motivates you to continuously improve in your field?
19. How do you set goals for yourself, and what strategies do you use to achieve them?
20. Describe a situation where you had to step outside your comfort zone to achieve a goal.
```